In [87]:
import pandas as pd
import numpy as np
import re 
import json
import datetime as dt

In [88]:
def get_competence(series):
  list_comp = []  
  for i in series:
    comp = re.sub("\n", "", str(i))
    list_comp.append(comp)

  return(list_comp)

def salaire(series, mode):
  # Definition de l'index pour le split des salaires min ou max
  if mode == "minimum":
    index = 0
  if mode == "maximum":
    index = 1
  
  # Identification si le salaire est present, et extraction le cas échéant
  if '€' in re.findall('€', str(series)):
    salaire = re.sub('\W+', '', series[1].split(" - ")[index].split(",")[0])
    salaire = int(re.sub(r"[^Z0-9]+",'', salaire))
  else:
    salaire = np.nan

  return(salaire)

In [89]:
data = pd.read_json('data.json')
df = data.copy()
df.dtypes

Intitulé du poste      object
Date de publication    object
lieu                   object
competences            object
Type de poste          object
dtype: object

In [90]:
df.shape

(230, 5)

In [91]:
df['Intitulé du poste'] = [' '.join(item).replace('\n', '').lower() for item in df['Intitulé du poste']]
df['lieu'] = [' '.join(item).replace('\n', '').lower() for item in df['lieu']]
df['competences'] = df['competences'].apply(get_competence).apply(lambda x: ' '.join(x))
df['Type de poste'] = [''.join(item).replace('\n', ', ').replace(', , ', '').lower().strip() for item in df['Type de poste']]
df

,Intitulé du poste,Date de publication,lieu,competences,Type de poste
0,data analyst - publicité,postée il y a 21 jours,paris,sql t-sql outils,"sept lieues sas, cabinet de recrutement, cdi -..."
1,data analyst - h/f,postée il y a 1 mois,"paris, la défense",support outils agile design big data,"edf france, client final, cdi - temps plein,"
2,customer data analyst (h/f),postée il y a 27 jours,paris,crm data quality sql sas python,"hermès, client final, cdi - temps plein,"
3,master data analyst h/f,postée il y a 21 jours,moussy-le-vieux,si support sap,"tereos, client final, cdi - temps plein,"
4,portfolio data analyst h/f,postée il y a 3 jours,france,outils support decisionnel sql test,"safran-group, client final, cdi - temps plein,"
...,...,...,...,...,...
225,sap fiori developer (h/f),postée il y a 27 jours,saint-ouen,sap abap progiciel système support,"groupe engie, client final, cdi - temps plein,"
226,engie digital - architecte données (h/f),postée il y a 3 jours,paris,digital outils securité sécurité t-sql,"groupe engie, client final, cdi - temps plein,"
227,quality assurance - lead qa - lead acceptance h/f,postée il y a 2 heures,boulogne-billancourt,test serveur jquery mobile tdd,"francaise des jeux, client final, cdi - temps ..."
228,data scientist,postée il y a 16 jours,"neuilly sur seine 38.000,00 € - 50.000,00 € / an",nosql unix java machine learning linux,"beezen, esn, cdi - temps plein,"


In [92]:
df['salaire_minimum'] = df['lieu'].apply(salaire, mode="minimum")
df['salaire_maximum'] = df['lieu'].apply(salaire, mode="maximum")
df

ValueError: invalid literal for int() with base 10: ''

In [ ]:
y = [' '.join(element.split()) for element in df['Date de publication']]

In [ ]:
date0 = dt.date(2023, 1, 15)
dates = []
for item in y:
    delai = item.split()
    if delai[-1] == "jours":
        delta = dt.timedelta(days=int(delai[-2]))
        date = date0 - delta
        dates.append(date)
    elif delai[-1] == 'mois':
        delta = dt.timedelta(weeks=4*int(delai[-2]))
        date = date0 - delta
        dates.append(date)
    elif delai[-1] == 'heures':
        delta = dt.timedelta(hours=int(delai[-2]))
        date = date0 - delta
        dates.append(date)
    elif delai[-1] == 'hier':
        date = dt.date(2013, 1, 14)
        dates.append(date)
df['Date de publication'] = dates
df

,Intitulé du poste,Date de publication,lieu,competences,Type de poste
0,data analyst - publicité,2022-12-25,paris,sql t-sql outils,"sept lieues sas, cabinet de recrutement, cdi -..."
1,data analyst - h/f,2022-12-18,"paris, la défense",support outils agile design big data,"edf france, client final, cdi - temps plein,"
2,customer data analyst (h/f),2022-12-19,paris,crm data quality sql sas python,"hermès, client final, cdi - temps plein,"
3,master data analyst h/f,2022-12-25,moussy-le-vieux,si support sap,"tereos, client final, cdi - temps plein,"
4,portfolio data analyst h/f,2023-01-12,france,outils support decisionnel sql test,"safran-group, client final, cdi - temps plein,"
...,...,...,...,...,...
225,sap fiori developer (h/f),2022-12-19,saint-ouen,sap abap progiciel système support,"groupe engie, client final, cdi - temps plein,"
226,engie digital - architecte données (h/f),2023-01-12,paris,digital outils securité sécurité t-sql,"groupe engie, client final, cdi - temps plein,"
227,quality assurance - lead qa - lead acceptance h/f,2023-01-15,boulogne-billancourt,test serveur jquery mobile tdd,"francaise des jeux, client final, cdi - temps ..."
228,data scientist,2022-12-30,"neuilly sur seine 38.000,00 € - 50.000,00 € / an",nosql unix java machine learning linux,"beezen, esn, cdi - temps plein,"


In [ ]:
# For the column of 'Type de contra' et 'Nom de la société' from original coulmn 'Type de poste'
df[['Nom de la société','Type de la société', 'Type de contrat']] = df["Type de poste"].str.split(",", expand = True, n=2)

df[['Type de contrat', 'temps travail_1']] = df["Type de contrat"].str.split("-", expand = True, n=2)
df[['Type de contrat', 'temps travail_2']] = df["Type de contrat"].str.split("/", expand = True, n=1)

df["Type de contrat"] = [''.join(item).replace(',', '').replace(' ', '') for item in df['Type de contrat']]

#Remove the 3 columns: 'temps travail_1' and 'temps travail_2','Type de la société'
df = df.drop(columns=['temps travail_1', 'temps travail_2', 'Type de la société'])
df

,Intitulé du poste,Date de publication,lieu,competences,Type de poste,salaire_min,salaire_max,Nom de la société,Type de contrat
0,data analyst - publicité,2022-12-25,[\nParis\n],sql t-sql outils,"sept lieues sas, cabinet de recrutement, cdi -...",NaN,NaN,sept lieues sas,cdi
1,data analyst - h/f,2022-12-18,"[\nParis, La Défense\n]",support outils agile design big data,"edf france, client final, cdi - temps plein,",NaN,NaN,edf france,cdi
2,customer data analyst (h/f),2022-12-19,[\nParis\n],crm data quality sql sas python,"hermès, client final, cdi - temps plein,",NaN,NaN,hermès,cdi
3,master data analyst h/f,2022-12-25,[\nMoussy-le-Vieux\n],si support sap,"tereos, client final, cdi - temps plein,",NaN,NaN,tereos,cdi
4,portfolio data analyst h/f,2023-01-12,[\nFrance\n],outils support decisionnel sql test,"safran-group, client final, cdi - temps plein,",NaN,NaN,safran-group,cdi
...,...,...,...,...,...,...,...,...,...
225,sap fiori developer (h/f),2022-12-19,[\nSaint-Ouen\n],sap abap progiciel système support,"groupe engie, client final, cdi - temps plein,",NaN,NaN,groupe engie,cdi
226,engie digital - architecte données (h/f),2023-01-12,[\nParis\n],digital outils securité sécurité t-sql,"groupe engie, client final, cdi - temps plein,",NaN,NaN,groupe engie,cdi
227,quality assurance - lead qa - lead acceptance h/f,2023-01-15,[\nBoulogne-Billancourt\n],test serveur jquery mobile tdd,"francaise des jeux, client final, cdi - temps ...",NaN,NaN,francaise des jeux,cdi
228,data scientist,2022-12-30,"[\nNEUILLY SUR SEINE\n, \n38.000,00 € - 50.000...",nosql unix java machine learning linux,"beezen, esn, cdi - temps plein,",38000.0,50000.0,beezen,cdi


In [ ]:
lieu = [item.split() for item in df['lieu']]

liste_lieu = []
for item in lieu:
    print(item)
    if len(item) > 5:
        liste_lieu.append(' '.join(item).split(',')[0])
    else:
        liste_lieu.append(' '.join(item))

df['lieu'] = liste_lieu
df

['paris']
['paris,', 'la', 'défense']
['paris']
['moussy-le-vieux']
['france']
['maisons', 'alfort', '45.000']
['paris']
['paris', '60.000']
['paris']
['rueil-malmaison']
['paris']
['la', 'defense']
['suresnes']
['saint-ouen']
['rueil', 'malmaison', '60.000']
['puteaux']
['paris', '60.000']
['paris']
['fontenay-sous-bois']
['gennevilliers', '60.000']
['paris']
['boulogne-billancourt', '55.000']
['saint-germain-en-laye']
['saint-germain-en-laye']
['suresnes', '70.000']
['suresnes', '70.000']
['la', 'défense']
['paris,', 'la', 'défense']
['rueil', 'malmaison', '35.000']
['ile-de-france']
['paris']
['ile', 'de', 'france']
['rueil-malmaison']
['ile', 'de', 'france']
['paris,', 'la', 'défense']
['val-de-marne']
['paris']
['paris']
['paris', '45.000']
['la', 'defense']
['massy']
['fontenay-sous-bois']
['paris']
['paris', '50.000']
['noisiel']
['île-de-france']
['la', 'defense']
['saint-ouen']
['île-de-france']
['colombes']
['levallois-perret']
['hauts-de-seine']
['la', 'defense']
['paris', '

,Intitulé du poste,Date de publication,lieu,competences,Type de poste
0,data analyst - publicité,postée il y a 21 jours,paris,sql t-sql outils,"sept lieues sas, cabinet de recrutement, cdi -..."
1,data analyst - h/f,postée il y a 1 mois,"paris, la défense",support outils agile design big data,"edf france, client final, cdi - temps plein,"
2,customer data analyst (h/f),postée il y a 27 jours,paris,crm data quality sql sas python,"hermès, client final, cdi - temps plein,"
3,master data analyst h/f,postée il y a 21 jours,moussy-le-vieux,si support sap,"tereos, client final, cdi - temps plein,"
4,portfolio data analyst h/f,postée il y a 3 jours,france,outils support decisionnel sql test,"safran-group, client final, cdi - temps plein,"
...,...,...,...,...,...
225,sap fiori developer (h/f),postée il y a 27 jours,saint-ouen,sap abap progiciel système support,"groupe engie, client final, cdi - temps plein,"
226,engie digital - architecte données (h/f),postée il y a 3 jours,paris,digital outils securité sécurité t-sql,"groupe engie, client final, cdi - temps plein,"
227,quality assurance - lead qa - lead acceptance h/f,postée il y a 2 heures,boulogne-billancourt,test serveur jquery mobile tdd,"francaise des jeux, client final, cdi - temps ..."
228,data scientist,postée il y a 16 jours,neuilly sur seine 38.000,nosql unix java machine learning linux,"beezen, esn, cdi - temps plein,"


In [ ]:
df = df.drop('Type de poste', axis=1)
df

,Intitulé du poste,Date de publication,lieu,competences,Nom de la société,Type de contrat,salaire_minimum,salaire_maximum
5,data analyst bo/qlick sense (h/f),2022-12-31,maisons alfort,support sql data management business objects t...,fed it,cdd,45000.0,50000.0
7,data analyst / chef de projet bi (h/f),2022-12-18,paris,data management business intelligence bases de...,fed it,cdi,60000.0,62000.0
14,data scientist - cdi,2022-12-18,rueil malmaison,marketing data management support,fed it,cdi,60000.0,65000.0
16,consultant data h/f,2023-01-13,paris,big data oracle cloud sap python,selescope,,60000.0,70000.0
19,chef de projets data - analytics h/f,2023-01-10,gennevilliers,marketing outils digital ux mobile,hays france,cdi,60000.0,70000.0
21,responsable etudes et projets si h/f,2023-01-15,boulogne-billancourt,si data management data quality méthodologie o...,michael page,cdi,55000.0,60000.0
24,ingénieur d’affaires / business developer–data...,2022-12-17,suresnes,business intelligence crm intelligence artific...,selescope,,70000.0,100000.0
25,business developer – ia / data analytics h/f,2022-12-27,suresnes,intelligence artificielle business intelligenc...,selescope,,70000.0,100000.0
28,business analyst / business intelligence,2013-01-14,rueil malmaison,business intelligence marketing intelligence a...,fed it,cdi,35000.0,45000.0
38,data scientist,2022-12-17,paris,machine learning méthodologie agile,key consulting,cdi,45000.0,70000.0


In [ ]:
# Nettoyage de la colonne Intitulé de poste, mode = data scientist
liste_postes = []
postes = [item for item in df['Intitulé du poste'].str.strip()]
for item in postes:
    if 'business' in item:
        liste_postes.append('business analyst')
    elif 'analyst' in item:
        liste_postes.append('data analyst')
    elif 'scientist' in item:
        liste_postes.append('data scientist')
    elif 'data engineer' in item:
        liste_postes.append('data engineer')
    elif 'chef de projet' in item:
        liste_postes.append('chef de projet')
    elif 'ingénieur' in item:
        liste_postes.append('engineer')
    elif 'architect' in item:
        liste_postes.append('architect')
    elif 'developer' in item:
        liste_postes.append('developer')
    elif 'développeur' in item:
        liste_postes.append('developer')
    elif 'developpeur' in item:
        liste_postes.append('developer')
    elif 'expert' in item:
        liste_postes.append('expert')
    elif 'consultant' in item:
        liste_postes.append('consultant')
    else:
        liste_postes.append('data scientist')
    
df['Intitulé du poste'] = liste_postes
df['Intitulé du poste'].value_counts()

engineer            17
data scientist       7
data analyst         4
business analyst     4
chef de projet       3
consultant           2
architect            2
developer            2
expert               1
Name: Intitulé du poste, dtype: int64

In [ ]:
# formatage du nom de l'entreprise
names = [item for item in df['Nom de la société']]
names
clean_names = []
for name in names:
    if 'societe general' in name:
        clean_names.append('societe general')
    else:
        clean_names.append(name)
df['Nom de la société'] = clean_names

In [ ]:
df.to_csv('clean_data.csv')